Her starter PINN delen

vi kunne godt tænke os en løsning til følgende problem

$$u''(t) = 0 \,\,\, u(0) = 1\,\,\, u'(1) = 2$$



In [160]:
import numpy as np
import matplotlib.pyplot as plt


import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split



def u_exact(t):
    return(2*t+1)

def u_dif_dif(t):

    return(torch.zeros(len(t)).view(-1,1) )


boundary = {'BC1': torch.tensor(1), 'BC2':torch.tensor(2)} 



##vi mangler u_diff til boundary


In [161]:
def loss_domain(t): #input vektor af tidpunkter

    u_pred = model(t)  #fx 10 punkter i y_pred
    u_pred_2 = u_pred.mean()
    
    first_derivative = torch.autograd.grad(u_pred_2, t, create_graph=True)[0]
    # We now have dloss/dx
    second_derivative = torch.autograd.grad(first_derivative.mean(), t)[0]
    # This computes d/dx(dloss/dx) = d2loss/dx2


    u_pred_dif_dif = second_derivative
    print("Den første ", (u_pred_dif_dif))
    print("Den anden ", (u_dif_dif(t)))
    
    print("Den næste: ",((u_pred_dif_dif-u_dif_dif(t))**2))
    return torch.mean((u_pred_dif_dif-u_dif_dif(t))**2)

def loss_boundary(t): #TODO : refaktor vektor
    

    u_pred_0 = model(torch.tensor(0.,requires_grad=True).view(-1,1))
    
    t1 = torch.tensor([0.,1.,2.] ,requires_grad=True).view(-1,1)
    u_pred_1 = model(t1)
    print(u_pred_1)

    
    u_pred_1 = torch.autograd.grad(u_pred_1, t1, create_graph=False,allow_unused=True)[0]
    print(u_pred_0)
    print(u_pred_1)
    loss = 1/2*((u_pred_0 - boundary['BC1'])**2 + (u_pred_1- boundary['BC2'])**2)
    return loss


def combine_loss(t):

    return loss_domain(t)+loss_boundary(t)






In [162]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(1, 64), # Input layer with 1 input and 64 neurons
            # nn.LeakyReLU(),        # Activation function
            # nn.Tanh(),
            # nn.ReLU(),
            nn.ELU(),
            nn.Linear(64, 64),# Hidden layer with 64 neurons
            # nn.LeakyReLU(),        # Activation function
            # nn.Tanh(),
            # nn.ReLU(),
            nn.ELU(),
            nn.Linear(64, 1)  # Output layer with 1 output
        )
    
    def forward(self, x):
        return self.network(x)
    
    

In [163]:
x_train = torch.linspace(-2, 5, 200).view(-1,1)
x_train.requires_grad = True


In [164]:
# Instantiate the model
model = PINN()

# Loss function
criterion = combine_loss

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [165]:
epochs = 4000
model.train()
for epoch in range(epochs):
    optimizer.zero_grad()           # Clear gradients for the next train
           
    loss = criterion(x_train) # Compute loss
    loss.backward()                 # Backward pass: Compute gradient of the loss with respect to model parameters
    optimizer.step()                # Update weights
    
    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')


Den første  tensor([[ 6.8513e-08],
        [ 6.1278e-08],
        [ 5.3826e-08],
        [ 4.6151e-08],
        [ 3.8244e-08],
        [ 3.0096e-08],
        [ 2.1700e-08],
        [-8.7018e-08],
        [-9.3496e-08],
        [-1.0020e-07],
        [-1.6146e-07],
        [-1.7157e-07],
        [-1.8205e-07],
        [-1.1486e-07],
        [-1.3137e-07],
        [-1.4833e-07],
        [-2.4146e-07],
        [-2.6016e-07],
        [-2.7931e-07],
        [-2.9895e-07],
        [-2.7383e-07],
        [-2.9202e-07],
        [-3.2357e-07],
        [-3.4195e-07],
        [-5.1596e-07],
        [-5.2222e-07],
        [-5.3653e-07],
        [-5.4785e-07],
        [-1.3371e-06],
        [-1.2101e-06],
        [-1.2140e-06],
        [-1.2188e-06],
        [-1.5393e-06],
        [-1.5411e-06],
        [-1.4946e-06],
        [-1.4670e-06],
        [-1.4130e-06],
        [-1.3846e-06],
        [-1.3571e-06],
        [-1.3324e-06],
        [-1.3065e-06],
        [-4.2110e-07],
        [-3.9062e-07],

RuntimeError: grad can be implicitly created only for scalar outputs